# Cropped Cam Sender

Grab video from a webcam, crop the frames, render and send, this commuicates to the Streams application via Kafka.

## Overview 

The file sends frames via the Kafka to the Streams application. One problem with demoning Streams has always been accesing live timly content. Video actually, WebCam video is a great source of such content, it's live, it's timely 
and it's ubiquitous. This notebook had a number of WebCam collections that we are used to provide video frames
to the Streams application that processes.

Their is no Streams code in this notebook, images are transmitted via Kafka. This notebook focusus on shredding the WebCam video, cropping it and sending it via Kafka. 

This file includes a section on setting up a MessageHub instance on the cloud to move the video datas


### Extras - 
The [croppedCamSender](croppedCamSender.ipynb) walks though pulling frames from internet webcams. Much more interesting than static files videos from the internet archive.

## Prerequisites 

This file requires the prerequites described in [README](/README.jupyter-py36.ipynb) be satisfied before procedi


In [1]:
import string
import os
import sys
import json
import base64
import ssl
import time
import datetime
import io
import logging
import re
import collections
import threading

import cv2
import requests
import urllib
import m3u8
from urllib.parse import urlparse
from PIL import Image
import numpy as np

import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import clear_output

from interactivecrop.interactivecrop import main as crop

if '../scripts' not in sys.path:
    sys.path.insert(0, '../scripts')

import stream_video as video
import credential

## Tutorials 
A very good walk through on the .m3u8 files that index .ts files that have 
the vido chunks that we need 

https://www.youtube.com/watch?v=bytnxnZFLeg

This is the cropping tool...

https://openbits.app/posts/python-interactive-cropping/


## test file access
The .ts files have the images to analyize. You need to walk through the index to get to the data. Mulitple index formats. Not sure if thier standard method for the schema, may make it too easy to 'borrow' someone ele's content.


"blob:https://www.earthcam.com/00631398-95e6-40df-9d75-71501c1a6da6"

<video muted="muted" crossorigin="anonymous" id="videoPlayer_html5_api" class="vjs-tech" preload="none" style="position: absolute" tabindex="-1" src="blob:https://www.earthcam.com/00631398-95e6-40df-9d75-71501c1a6da6"></video>



# Locating chunks of video frames.



The live webcams have a hierachy of files that terminates with files with chunks of frames, .ts files. 
The deepest .m3u8 will be updated with every new chunk's arrival. Work your way down to the lowests m3u8 file handle, this file is newly captured .ts files, it gets rewritten with new video chunks arrive. 


##  examples

Below is collection of sites that we can fetch video via .m3u8 that reference .ts files. 

### whatsupcams 

The standard process is to find a webcam that you are interested in getting the contents from, using 
Chome to look a the network traffic looking for files being fetched that end inf '.m3u8', repeatily. 
Look at the preview, you have the correct file if you see content refering to .ts files like this...
```text
#EXTM3U
#EXT-X-VERSION:3
#EXT-X-MEDIA-SEQUENCE:4982
#EXT-X-TARGETDURATION:12
#EXTINF:11.989,
hr_zadar4-4982.ts
#EXTINF:11.986,
hr_zadar4-4983.ts
#EXTINF:11.989,
hr_zadar4-4984.ts
```


Capture the url link to the file in chrome and past into M3U8 variable in below. 
https://cdn-004.whatsupcams.com/hls/hr_zadar4.m3u8

Below you will find the video link and M3U8 for webcam at different sites


| url | M3U8 |
|:-----|:-----|
|https://www.whatsupcams.com/en/webcams/croatia/zadar/zadar/zadar-narodni-trg-peoples-square/| https://cdn-004.whatsupcams.com/hls/hr_zadar4.m3u8|
| https://www.whatsupcams.com/en/webcams/italy/veneto/venice/webcam-venice-rialto-bridge/ | https://cdn-001.whatsupcams.com/hls/it_venezia01.m3u8| 
| https://www.whatsupcams.com/en/webcams/croatia/zadar/zadar/zadar-trg-petra-zoranica/ | https://cdn-001.whatsupcams.com/hls/hr_zadar3.m3u8 |

        

### caltrans

| url | M3U8 |
|:-----|:-----|
|http://cwwp2.dot.ca.gov/vm/loc/d4/tv388sr1justsouthofpresidiotunnel.htm| http://wzmedia.dot.ca.gov/D4/N1_at_Presidio_Tunnel.stream/chunklist_w665039362.m3u8 |
| http://cwwp2.dot.ca.gov/vm/loc/d4/tv457sr92elcaminoreal.htm | http://wzmedia.dot.ca.gov/D4/W92_at_El_Camino_Real.stream/chunklist_w2118793021.m3u8 |
|http://cwwp2.dot.ca.gov/vm/loc/d4/tve83us101i580.htm | http://wzmedia.dot.ca.gov/D4/N101_at_580.stream/chunklist_w987073694.m3u8 |
| http://cwwp2.dot.ca.gov/vm/loc/d4/tv302us10117thstreet.htm | http://wzmedia.dot.ca.gov/D4/S101_at_17th_St.stream/chunklist_w1949748683.m3u8 |


- Go to the map page, http://cwwp2.dot.ca.gov/vm/iframemap.htm
- select a video camera from the map
- select 'Open this camera in a bookmark-friendly format"
- Using Chrome, grap URL of opened page, for example (http://cwwp2.dot.ca.gov/vm/loc/d4/tv388sr1justsouthofpresidiotunnel.htm). Access the developer tool (View | Developer | Developer Tools) the  'Network' tab
 - Find a 'chunklist*.m3u8' file, the contents of which look like.
 ```
#EXTM3U
#EXT-X-VERSION:4
#EXT-X-TARGETDURATION:11
#EXT-X-MEDIA-SEQUENCE:7817
#EXTINF:9.61,
media_w1296425575_7817.ts
#EXTINF:10.677,
media_w1296425575_7818.ts
#EXTINF:9.61,
media_w1296425575_7819.ts
```
Assign this value to **M3U8** below


Notes : 
- I have found that the caltrans camera are sporatic.

## Huntington beach 
This site has multiple cameras on one page, bring up the page in Chrome's "Network" developer tab. Find the 'chunklist_\*.m3u8' file and set it to the **M3U8** file.

| url | M3U8 |
|:-----|:-----|
| https://hbcams.com/ | https://56cdb389b57ec.streamlock.net:1935/live/hbnorth.stream/chunklist_w790486524.m3u8 |
| | https://56cdb389b57ec.streamlock.net:1935/live/hbsouth.stream/chunklist_w1444292277.m3u8 |

## Misc
|Note |url | M3U8 |
|:----|:---|:---- |
|Golf by the sea |https://www.cbk-haylingisland.com/webcam/ | https://ngx.cr2.streamzilla.xlcdn.com/session/61110a777793643c03cc9b899f57c76c/sz/streamdays/wowza4/live/hayling-kitesurfing/chunklist.m3u8     |

# Define cropping region for live video streams

The rest of the notebook walks through 
- collecting some sample frames
- selecting frame for cropping 
- cropping the frame
- reviewing the specified cropped region 
- finally streaming the video and rendering the frames cropped.



### Assign the url of the chunklist to the  M3U8 varible below. 

Select the feed that you want to extract frames from by assigning the **M3U8** variable.


In [ ]:
camList = [
    "http://wzmedia.dot.ca.gov/D4/N1_at_Presidio_Tunnel.stream/chunklist_w665039362.m3u8",
"http://wzmedia.dot.ca.gov/D4/W92_at_El_Camino_Real.stream/chunklist_w2118793021.m3u8",
"http://wzmedia.dot.ca.gov/D4/S101_at_17th_St.stream/chunklist_w1949748683.m3u8",
"http://wzmedia.dot.ca.gov/D4/N101_at_580.stream/chunklist_w987073694.m3u8",
"https://56cdb389b57ec.streamlock.net:1935/live/hbnorth.stream/chunklist_w790486524.m3u8",
"https://56cdb389b57ec.streamlock.net:1935/live/hbnorth.stream/chunklist_w1235916242.m3u8",
"https://56cdb389b57ec.streamlock.net:1935/live/hbsouth.stream/chunklist_w1444292277.m3u8",
"https://cdn-004.whatsupcams.com/hls/hr_zadar4.m3u8",
"https://cdn-001.whatsupcams.com/hls/hr_zadar3.m3u8",
"https://cdn-001.whatsupcams.com/hls/it_venezia01.m3u8",
"https://ngx.cr2.streamzilla.xlcdn.com/session/61110a777793643c03cc9b899f57c76c/sz/streamdays/wowza4/live/hayling-kitesurfing/chunklist.m3u8"
          ]           
cameraRadio = widgets.RadioButtons(
    options=camList,
     value=camList[9],
    description="<b>Select<br>WebCam <br> to <br> Stream</b>"
)
display(cameraRadio)


## Collect a set of sample frames

Collect some frames to be used for defining the crop region.

In [ ]:
M3U8 = cameraRadio.value
#cams_chunk_url = M3U8
cams_base = video.extract_base(M3U8)
CROP_FRAMES = video.collect_frames(M3U8, 2)
print("\n\nCollected {} frames".format(len(CROP_FRAMES)))

## Select a frame to define the crop region

### Use  'crop'  button to select the frame

In [ ]:

def frame_selected(idx):
    global FRAME_IDX
    print("Use frame {} to work out cropping region.".format(idx))
    FRAME_IDX = idx
    
imgsel = video.imageSelect(CROP_FRAMES)
imgsel.ignition(frame_selected)

## Define crop region using the selected image.
### Use  'Save Crop Sizes' when you're satisfied with the crop region.

In [ ]:
FRAME_IDX = 1
SHAPE = None
def grabCropShape(image_name, shape):
    global SHAPE
    SHAPE = shape
    print("set SHAPE ", image_name, shape, flush=True)

try :
    print("Using frame #{} to specify crop region.".format(FRAME_IDX))
except NameError as e:
    print("\n\nERROR * You must select the 'crop' button above to commit your selection.\n\n")


crop([CROP_FRAMES[FRAME_IDX]], callback=grabCropShape)


## Verify the cropped region to on the collected frames 

In [ ]:
if SHAPE is None:
    print("\n\nYou must select the 'Save Crop Sizes' button above to commit crop region.\n\n")
else:
    print("Using shape : {}".format(SHAPE))
    video.collected_frames(CROP_FRAMES, SHAPE)

## Dashboard

Collect frames from the video feed, crop and display.

Threaded to get the button pushes. 

In [ ]:
creds = json.loads(credential.magsEventStream)
producer = video.kafka_producer(creds,)

scv = video.streamCroppedVideo(M3U8, SHAPE, 
                           wait_time=2, 
                           frame_modulo=6, 
                           kafka_producer=producer, 
                           kafka_topic='VideoFrame')
scv.ignition(True)
print("done")

In [ ]:
scv.ignition(False)
print("done")

## end

In [ ]:
scv.execute.is_set()

In [ ]:
scv.execute.clear()